# Inits

In [1]:
!pip3 install nltk --user
!pip3 install fuzzywuzzy[speedup] --user

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
from tqdm import tqdm
import difflib

In [3]:
# these needed to be installed
import nltk.data
from fuzzywuzzy import fuzz

In [4]:
import os
import re, string

def ls(path):
    ls = [f for f in os.listdir(path)]
    print(ls)
    return ls
    
def cd(path):
    os.chdir(path)
    cwd = os.getcwd()
    print("In directory:", cwd)

print("Saved models:")
dirs = ls("/home/kangeunsu/word-rnn-tensorflow")

print("\n")
for dir in dirs:
    if "save" in dir:
        print(dir)
    
print("\n")
print("Datasets:")
ls("/home/kangeunsu/word-rnn-tensorflow/data")

cd("/home/kangeunsu/word-rnn-tensorflow")

Saved models:
['sample.py', 'save', 'model.py', 'save_testrun', '__pycache__', '.git', 'output_america_9k', 'beam.pyc', 'how_to.txt', 'GENERATED__save_100ep_korea_plus_trump_21456_longer_than_400chars.txt', 'last_plot.png', 'README.md', 'beam.py', 'output_china', '.travis.yml', 'output_america', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs.txt', 'june15-2018', 'save_first_decent_model_subdataset', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs__distances.png', '.gitignore', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs__distances.pdf', 'logs', 'save_korea_texts_3064_longer_than_400chars__W16Beam_paragraphs.txt', 'save_korea_texts_3064_longer_than_400chars', 'model.pyc', 'train.py', 'save_50ep_korea_texts_3064_longer_than_400chars', 'save_100ep_korea_plus_trump_21456_longer_than_400chars', 'save_first_decent_model_subdataset__W4Beam_paragraphs.txt', 'save_test_run', 'save_50ep_korea_plus_trump_21456_longer_than_400chars', 'd

In [5]:
# These finished:
"save_korea_texts_3064_longer_than_400chars"
"save_100ep_korea_minus_trump_2326_longer_than_400chars"
"save_korea_texts_3064_longer_than_400chars_128x2M"

"save_50ep_korea_plus_trump_21456_longer_than_400chars"
"save_100ep_korea_plus_trump_21456_longer_than_400chars"



'save_100ep_korea_plus_trump_21456_longer_than_400chars'

# Load Generated

In [6]:
ls("/home/kangeunsu/word-rnn-tensorflow/")

"""
save_first_decent_model_subdataset__noBeam_paragraphs.txt
save_first_decent_model_subdataset__W4Beam_paragraphs.txt
save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs.txt
save_korea_texts_3064_longer_than_400chars__noBeam_paragraphs.txt
save_korea_texts_3064_longer_than_400chars__W16Beam_paragraphs.txt
save_korea_texts_3064_longer_than_400chars__W4_60paragraphs.txt

GENERATED__save_100ep_korea_plus_trump_21456_longer_than_400chars.txt
"""

file = "GENERATED__save_100ep_korea_plus_trump_21456_longer_than_400chars.txt"
with open(file) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
content = "\n".join(content)

#content = content.decode('utf-8')


['sample.py', 'save', 'model.py', 'save_testrun', '__pycache__', '.git', 'output_america_9k', 'beam.pyc', 'how_to.txt', 'GENERATED__save_100ep_korea_plus_trump_21456_longer_than_400chars.txt', 'last_plot.png', 'README.md', 'beam.py', 'output_china', '.travis.yml', 'output_america', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs.txt', 'june15-2018', 'save_first_decent_model_subdataset', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs__distances.png', '.gitignore', 'save_korea_texts_3064_longer_than_400chars_128x2M__W460paragraphs__distances.pdf', 'logs', 'save_korea_texts_3064_longer_than_400chars__W16Beam_paragraphs.txt', 'save_korea_texts_3064_longer_than_400chars', 'model.pyc', 'train.py', 'save_50ep_korea_texts_3064_longer_than_400chars', 'save_100ep_korea_plus_trump_21456_longer_than_400chars', 'save_first_decent_model_subdataset__W4Beam_paragraphs.txt', 'save_test_run', 'save_50ep_korea_plus_trump_21456_longer_than_400chars', 'data', 'tests',

In [7]:
print("Generated",len(content),"letters....")
print(content[0:500])

Generated 334625 letters....
coming, nice and new and “smart!” You shouldn’t be partners with a Gas Killing Animal who kills his people and enjoys it! — Donald J. Trump (@realDonaldTrump) April 11, 2018 The White House did not immediately respond to a request for comment. The White House did not immediately respond to a request for comment. The White House said that the United States would prematurely withdraw from Syria,” the official said. At the end of the day, U.S. President Donald Trump said on Twitter that the U.S. wo


# Load source text

In [8]:
data_file = "/home/kangeunsu/word-rnn-tensorflow/data/korea_plus_trump_21456_texts_longer_than_400chars/"
ls(data_file)

whole_text_path = "/home/kangeunsu/word-rnn-tensorflow/data/korea_plus_trump_21456_texts_longer_than_400chars/input.txt"
with open(whole_text_path, 'r') as content_file:
    whole_text = content_file.read()
    
print("Loaded",len(whole_text),"chars", whole_text[0:50])

['vocab.pkl', 'data.npy', 'input.txt']
Loaded 122230800 chars Donald Trump proposes 'IQ tests' face-off with Sec


In [9]:
# Version which needs NLTK
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def string_to_sentences(paragraph):
    return tokenizer.tokenize(paragraph)
    # 106 481 sentences

In [28]:
# Version which doesn't need NLTK
#
#def string_to_sentences(paragraph):
#    sentenceEnders = re.compile('[.!?]')
#    sentenceList = sentenceEnders.split(paragraph)
#    return sentenceList
#    # 142 924 sentences

In [10]:
def string_to_words(s):
    return [re.sub('^[{0}]+|[{0}]+$'.format(string.punctuation), '', w) for w in s.split()]

#whole_text = whole_text.decode('utf-8')
    
full_word_list = string_to_words(whole_text)
full_sentence_list = string_to_sentences(whole_text)
#unique_words = list(set(word_list))
print(len(full_word_list),"words")
print(len(full_sentence_list),"sentences")
#print(len(unique_words),"unique", unique_words[0:100])

18929606 words
775965 sentences


In [13]:
idx = np.random.randint(1, len(full_sentence_list), size=3)
for i in idx:
    print(full_sentence_list[i])
    print("===")


Michael Cohen, Trump’s personal pit bull lawyer, is radioactive over his role in the Stormy Daniels affair and may face legal exposure of his own.
===
Both presidents used the military to perform another task.
===
Incumbents have a name identification advantage.
===


In [14]:
#sample_paragraph = "/n".join(paragraphs)
#sample_paragraph = paragraphs[0] + paragraphs[1] + paragraphs[2]
#sample_paragraph = sample_paragraph[0:200]
print("Working with file", file, "now...")
sample_paragraph = content

sample_sentences = string_to_sentences(sample_paragraph)
print("parsed generated text as", len(sample_sentences), " sentences.")

Working with file GENERATED__save_100ep_korea_plus_trump_21456_longer_than_400chars.txt now...
parsed generated text as 1540  sentences.


In [33]:
# first few:
#sample_sentences = sample_sentences[100:]

## Suspicious sentences detection

In [ ]:
closest_sentences = {}

similarities = []

i = 0
#for sentence in tqdm(sample_sentences):
for sentence in sample_sentences:
    closest_sentences[i] = []
    
    max_so_far = -1.0
    closest_sentence = ""
    
    for close_sentence in full_sentence_list:
        r = float(fuzz.ratio(sentence, close_sentence)) / 100.0
        # SUPER SLOW >> r = (difflib.SequenceMatcher(None, sentence, close_sentence).ratio())
        if r >= max_so_far:
            max_so_far = r
            closest_sentence = close_sentence
    
            closest_sentences[i].append([r, closest_sentence])
    similarities.append(max_so_far)
    
    ###
    """
    if max_so_far > 0.8:
        print(i)
        print("Generated sentence ",i," \"", sentence, "\" has closest:")
        print(max_so_far,":",closest_sentence)
        print("-----------------")
    """
    if max_so_far < 0.7:
        # print "good enough" sentences
        print(sentence)
    
    
    #for close in closest_sentences[i]:
    #    print(close[0],":", close[1])
    
    i = i+1


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(similarities)
plt.ylim(0.0,1.0)
plt.title("Levenstein distance to nearest sentence in the real dataset.")
# Two layer model with 256 neurons, beam search width 16
plt.xlabel("Default model and sampling.")
plt.show()

In [ ]:
fig.savefig("/home/kangeunsu/ArtML/_Word_rnn_analysis/last__distances.pdf", bbox_inches='tight')

### What are the best generated sentences?

In [ ]:
# min max score

tmp_scores = []
tmp_sentences = []
tmp_close_sentences = []
for i,sentence in enumerate(sample_sentences):
    
    max_so_far = -1.0
    maximum_sentence=""
    #print("Sentence ",i," \"", sentence, "\" has closest:")
    for close in closest_sentences[i]:
        #print(close[0],":", close[1])
        if close[0]>max_so_far:
            max_so_far=close[0] 
            maximum_sentence=sentence
            closest_sentence=close[1]
    
    tmp_scores.append(max_so_far)
    tmp_sentences.append(maximum_sentence)
    tmp_close_sentences.append(closest_sentence)
    #print("-----------------")
    
sort_order = np.argsort(tmp_scores)
tmp_scores = [tmp_scores[i] for i in sort_order]
tmp_sentences = [tmp_sentences[i] for i in sort_order]
tmp_close_sentences = [tmp_close_sentences[i] for i in sort_order]

print("Most innovative sentences are: (sorted)")
for i in range(len(tmp_scores)):
    print(tmp_scores[i], ":", tmp_sentences[i])
    print("~~~",tmp_close_sentences[i])
    print("")

In [ ]:
# Might need to restart after each call

#kernel.restart()
import os
os._exit(0)

# Other experiments

In [ ]:
These might need:
#pip3 install python-Levenshtein --user
#pip3 install nltk --user
#pip3 install Distance --user
#pip3 install jellyfish --user
#pip3 install fuzzywuzzy[speedup] --user

In [35]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print(tokenizer.tokenize("Sample sentence. Some text."))

s1 = "While you do not have a lot of different cases"
s2 = "The US does not have a diplomatic presence in North Korea"

import Levenshtein
print (Levenshtein.ratio(s1,s2))

import distance
print (distance.levenshtein(s1,s2))

import jellyfish
print (jellyfish.levenshtein_distance(s1,s2))

from fuzzywuzzy import fuzz
print (fuzz.ratio(s1,s2))
print (fuzz.token_sort_ratio(s1,s2))
print (fuzz.token_set_ratio(s1,s2))


import difflib
print (difflib.SequenceMatcher(None, s1,s2).ratio())

""

['Sample sentence.', 'Some text.']
0.5436893203883495
33
33
54
50
54
0.5048543689320388


''

In [12]:
# Default python library only...
# ... it's very slow though
s1 = "While you do not have a lot of different cases"
s2 = "The US does not have a diplomatic presence in North Korea"

import difflib
print (difflib.SequenceMatcher(None, s1,s2).ratio())

0.5048543689320388
